In [2]:
import numpy as np
import mido
import pretty_midi
import sklearn

In [3]:
def notes_to_instrument(notes, type_inst=0):
    instrument = pretty_midi.Instrument(type_inst)
    to_write = []
    for i in range(len(notes)):
        if notes[i] != -1:
            to_write.append(pretty_midi.Note(100,notes[i],i,i+1))
    instrument.notes = to_write
    return instrument

In [36]:
def calculate_hamming_distance(playing, potential):    
    diff = np.sum(np.abs(playing-potential))
    
    score = 10-(diff-1)**2
    
    return score

In [5]:
def make_histogram(melody):
    histogram = np.zeros(76-54)
    for note in melody:
        histogram[note-54] += 1
    return histogram/32

In [39]:
class Player():
    def __init__(self, instrument):
        self.instrument = instrument
        self.isMainVoice = False
        self.melody = np.random.randint(54, 76, 8)
        self.hist = make_histogram(self.melody)
        self.score = 0
        self.playing = None
    
    def set_other_players(self, others):
        self.others = others
        
    def become_main(self):
        self.isMainVoice = True
        self.playing = self
        self.score = calculate_hamming_distance(self.hist, self.hist)
        
        for player in self.others:
            player.set_playing(self)
    
    def set_playing(self, playing):
        self.playing = playing
        self.isMainVoice = False
    
    def adjust_melody(self):
        self.score = calculate_hamming_distance(self.playing.hist, self.hist)
        new_melody = np.copy(self.melody)
        for i in range(len(new_melody)):
            if np.random.random() < 0.1:
                new_melody[i] += np.random.randint(-3,4)
                new_melody[i] = max(min(75, new_melody[i]), 54)
        new_hist = make_histogram(new_melody)
        new_score = calculate_hamming_distance(self.playing.hist, new_hist)
        
        if new_score > self.score:
            self.melody = new_melody
            self.hist = new_hist 
            
    def update(self, can_change_main):
        if not self.isMainVoice:
            self.adjust_melody()
        elif can_change_main:
            highest_score = self.score
            best_player = self
            
            for player in self.others:
                if player.score > highest_score:
                    #print(player.score, highest_score)
                    highest_score = player.score
                    best_player = player
            
            print(self.hist, best_player.hist)
            best_player.become_main()

I'm doing this because that might be good for harmony

In [40]:
p1 = Player(1)
p2 = Player(57)
p3 = Player(40)

p1.set_other_players([p2, p3])
p2.set_other_players([p1, p3])
p3.set_other_players([p2, p1])

p1_out = []
p2_out = []
p3_out = []

p1.become_main()

players = [p1, p2, p3]
outputs = [p1_out, p2_out, p3_out]

time_since_last_change = 0
current_main = 0

for j in range(200):
    for i in range(len(players)):
        if players[i].isMainVoice:
            outputs[i].append(players[i].melody[j%len(players[i].melody)])
            if current_main == i:
                time_since_last_change += 1
            else:
                time_since_last_change = 0
                current_main = i
            #print(i, players[i].melody[j%len(players[i].melody)])
        else:
            outputs[i].append(-1)
    
    for player in players:
        player.update(time_since_last_change > 8)

midi = pretty_midi.PrettyMIDI()
p1_instrument = notes_to_instrument(p1_out, p1.instrument)
p2_instrument = notes_to_instrument(p2_out, p2.instrument)
p3_instrument = notes_to_instrument(p3_out, p3.instrument)
midi.instruments = [p1_instrument, p2_instrument, p3_instrument]
midi.write("Auctioning.mid")

[0.      0.03125 0.03125 0.      0.      0.      0.      0.      0.03125
 0.      0.      0.03125 0.03125 0.03125 0.      0.      0.      0.0625
 0.      0.      0.      0.     ] [0.      0.      0.      0.      0.      0.      0.      0.      0.
 0.      0.      0.0625  0.      0.      0.      0.      0.03125 0.
 0.0625  0.0625  0.      0.03125]
[0.      0.      0.      0.      0.      0.      0.      0.      0.
 0.      0.      0.0625  0.      0.      0.      0.      0.03125 0.
 0.0625  0.0625  0.      0.03125] [0.      0.      0.      0.      0.      0.      0.0625  0.      0.03125
 0.03125 0.03125 0.      0.0625  0.      0.      0.      0.      0.
 0.      0.      0.03125 0.     ]
[0.      0.      0.      0.      0.      0.      0.0625  0.      0.03125
 0.03125 0.03125 0.      0.0625  0.      0.      0.      0.      0.
 0.      0.      0.03125 0.     ] [0.      0.      0.      0.      0.      0.      0.0625  0.      0.03125
 0.03125 0.03125 0.      0.0625  0.      0.      0.      0